In [4]:
import cv2
import matplotlib.pyplot as plt
def view_clip(step):
    for i in range (300, 1100, step) :
        I = cv2.imread('pedestrian/input/in%06d.jpg' % i)
        cv2.imshow("I", I)
#         plt.imshow(I)
        cv2.waitKey (10)

view_clip(5)

In [1]:

import cv2

def view_clip(step):
    for i in range (300, 1100, step) :
        I = cv2.imread('pedestrian/input/in%06d.jpg' % i)
        cv2.imshow ("I",I)
        cv2.waitKey (10)

view_clip(5)
cv2.destroyAllWindows()

In [5]:
import cv2

def clip_binarize(folder):    
    prev = cv2.imread(f'{folder}/input/in000001.jpg', cv2.IMREAD_GRAYSCALE).astype('int')
    for i in range (300, 1100) :
        next = cv2.imread(f'{folder}/input/in%06d.jpg' % i, cv2.IMREAD_GRAYSCALE).astype('int')
        D = abs(next - prev)
        B = 1*( D > 10)
        B = (B * 255).astype('uint8')
        cv2.imshow("I", B)
        cv2.waitKey(10)
        prev = next
    cv2.destroyAllWindows()

clip_binarize('office')

In [8]:
import cv2
import numpy as np


def clip_morph(folder):    
    prev = cv2.imread(f'{folder}/input/in000001.jpg', cv2.IMREAD_GRAYSCALE).astype('int')
    for i in range (570, 2050): #for office
        next = cv2.imread(f'{folder}/input/in%06d.jpg' % i, cv2.IMREAD_GRAYSCALE).astype('int')
        D = abs(next - prev)
        B = 1*( D > 10)
        B = (B * 255).astype('uint8')
        B_blur = cv2.medianBlur(B, 5)
        B_erode = cv2.erode(B_blur, np.ones((3, 3)))
        B_dil = cv2.dilate(B_erode, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)), iterations=10)
        cv2.imshow("I", B_dil)
        cv2.waitKey(10)
        prev = next
    cv2.destroyAllWindows()

clip_morph('office')

In [3]:
import cv2
import numpy as np


def clip_final():    
    I = cv2.imread('pedestrian/input/in000001.jpg')
    prev = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY).astype('int')

    for i in range (300, 1100) :
        I_next = cv2.imread('pedestrian/input/in%06d.jpg' % i)
        next = cv2.cvtColor(I_next, cv2.COLOR_BGR2GRAY).astype('int')

        D = abs(next - prev)
        B = 1*( D > 13)
        B = (B * 255).astype('uint8')

        B_blur = cv2.medianBlur(B, 5)
        B_erode = cv2.erode(B_blur, np.ones((3, 3)))
        B = cv2.dilate(B_erode, np.ones((3, 3)), iterations=2)

        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(B)
        I_VIS = I_next # copy of the input image
        if ( stats . shape [0] > 1) : # are there any objects
            tab = stats [1: ,4] # 4 columns without first element
            pi = np . argmax ( tab )# finding the index of the largest item
            pi = pi + 1 # increment because we want the index in stats , not in tab
            # drawing a bbox
            if stats[pi, 4] > 100:
                cv2 . rectangle ( I_VIS ,( stats [pi ,0] , stats [pi ,1]) ,( stats [ pi ,0]+ stats [ pi ,2] , stats [ pi
                ,1]+ stats [ pi ,3]) ,(255 ,0 ,0) ,2)
                # print information about the field and the number of the largest element
                cv2.putText(I_VIS,"%f" % stats [pi ,4], (stats[pi, 0], stats[pi, 1]), cv2.
                    FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0))
                cv2 . putText ( I_VIS ,"%d" %pi ,(int( centroids [ pi ,0]) ,int( centroids [ pi ,1]) ) , cv2 .
                FONT_HERSHEY_SIMPLEX ,1 ,(255 ,0 ,0) )
            cv2.imshow("I", I_VIS)

        cv2.waitKey(10)
        prev = next

clip_final()
cv2.destroyAllWindows()

In [2]:
import cv2
import numpy as np


def clip_final(folder, iter):    
    I = cv2.imread(f'{folder}/input/in000001.jpg')
    prev = cv2.cvtColor(I, cv2.COLOR_BGR2GRAY).astype('int')

    TP, TN, FP, FN = 0, 0, 0, 0

    f = open(f'{folder}/temporalROI.txt','r')
    line = f.readline()
    roi_start, roi_end = line.split()   
    roi_start = int(roi_start)
    roi_end = int(roi_end)
    
    for i in range (roi_start, roi_end) :
        I_next = cv2.imread(f'{folder}/input/in%06d.jpg' % i)
        next = cv2.cvtColor(I_next, cv2.COLOR_BGR2GRAY).astype('int')

        D = abs(next - prev)
        B = 1*( D > 13)
        B = (B * 255).astype('uint8')

        B_blur = cv2.medianBlur(B, 5)
        B_erode = cv2.erode(B_blur, np.ones((3, 3)))
        B = cv2.dilate(B_erode, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)), iterations=iter)

        I_ground = cv2.imread(f'{folder}/groundtruth/gt%06d.png' % i)
        GTB = cv2.cvtColor(I_ground, cv2.COLOR_BGR2GRAY).astype('int')

        TP_M = np.logical_and((B == 255), (GTB == 255)) 
        TP_S = np.sum ( TP_M ) 
        TP = TP + TP_S 

        TN_M = np.logical_and((B == 0), (GTB == 0)) 
        TN_S = np.sum (TN_M) 
        TN += + TN_S 

        FP_M = np.logical_and((B == 255), (GTB == 0))
        FP_S = np.sum(FP_M)
        FP += FP_S

        FN_M = np.logical_and((B == 0), (GTB == 255))
        FN_S = np.sum(FN_M)
        FN += FN_S

        retval, labels, stats, centroids = cv2.connectedComponentsWithStats(B)
        I_VIS = I_next # copy of the input image
        if ( stats.shape[0] > 1) : # are there any objects
            tab = stats [1:, 4] # 4 columns without first element
            pi = np.argmax(tab)# finding the index of the largest item
            pi = pi + 1 

            if stats[pi, 4] > 100:
                cv2.rectangle(I_VIS, (stats[pi, 0], stats[pi, 1]), 
                              (stats[pi, 0] + stats[pi, 2], stats[pi, 1] + stats[pi, 3]), (255, 0, 0), 2)

                cv2.putText(I_VIS, "%f" % stats[pi, 4], (stats[pi, 0], stats[pi, 1]), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0))
                cv2.putText(I_VIS, "%d" %pi, (int(centroids[pi, 0]), int(centroids[pi, 1])), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0))
            cv2.imshow("I", I_VIS)

        cv2.waitKey(10)
        prev = next
    precission = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precission * recall / (precission + recall)
    print(f"Clip Name: {folder}\nPrecission: {precission} Recall: {recall} F1: {f1}")
    cv2.destroyAllWindows()




In [6]:
clip_final('pedestrian', 2)
# clip_final('office', 10)
# clip_final('highway', 2)

Clip Name: pedestrian
Precission: 0.705856503946413 Recall: 0.6201409529828787 F1: 0.6602283024587029
